# Advanced Swarmpy

<a href="https://www.python.org/downloads/release/python-3100/" 
target="_blank"><img src="https://img.shields.io/badge/python-3.10-blue.svg" alt="Python Version" /></a>

___

Combining pipeline, masks and ant parameter randomization

The goal here is to warmstart the pheromone trails by applying ACO to a partition of the graph. Therefore we use intermediate pipelines with mask and eventually apply iterations on the entire problem

In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
os.chdir('..')
from copy import deepcopy

from swarmpy_tsp import *
import plotly.express as px


[SwarmPy] NumExpr defaulting to 8 threads.


In [2]:
G1, opt_score = Antcoder('test_set/berlin52')

In [3]:
mask = np.ones_like(G1['e'], dtype=bool)
mid = int(G1['e'].shape[0]/2)
mask[:mid, :mid] = 0

In [6]:
scores=[]
for _ in range(10):
    G = deepcopy(G1)

    aco_with_Daemon1 = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0], ant_params={'mask':mask})),
            ("Sol", SolutionConstructor()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.4,.6])),
    ], iter_max=10, as_step=True
    )

    aco_with_Daemon2 = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0], ant_params={'mask':~mask})),
            ("Sol", SolutionConstructor()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.4,.6])),
            
    ], iter_max=10, as_step=True
    )

    aco_final = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0])),
            ("Sol", SolutionConstructor()),
            ("DA", DaemonActions()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.1,1])),
            
    ], iter_max=30, last_step=True
    )

    optimus_prime_pipeline = ACO_Pipeline([('a', aco_with_Daemon1), ('b',aco_with_Daemon2), ('d',aco_final)], iter_max=1, metapipeline=True)

    solutions_w_daemon = optimus_prime_pipeline.run(G=G,)
    scores_w_daemon = np.array([el[1] for el in solutions_w_daemon['solutions']]) - opt_score

    scores.append(scores_w_daemon)

SwarmPy |Step| Score : 5945.115881258639: 100%|██████████| 10/10 [00:01<00:00,  6.65it/s]
SwarmPy |Step| Score : 5854.271767621108: 100%|██████████| 10/10 [00:01<00:00,  8.66it/s]
SwarmPy |Final Step| Score : 7944.5012345336645: 100%|██████████| 30/30 [00:07<00:00,  3.90it/s]
SwarmPy | Score : 7944.5012345336645: 100%|██████████| 1/1 [00:10<00:00, 10.37s/it]
SwarmPy |Step| Score : 6098.86713016118: 100%|██████████| 10/10 [00:01<00:00,  7.86it/s]
SwarmPy |Step| Score : 5764.426116849343: 100%|██████████| 10/10 [00:01<00:00,  8.74it/s]
SwarmPy |Final Step| Score : 7764.96122527121: 100%|██████████| 30/30 [00:07<00:00,  4.17it/s]
SwarmPy | Score : 7764.96122527121: 100%|██████████| 1/1 [00:09<00:00,  9.62s/it]
SwarmPy |Step| Score : 5971.120325035439: 100%|██████████| 10/10 [00:01<00:00,  7.17it/s]
SwarmPy |Step| Score : 6084.243558495553: 100%|██████████| 10/10 [00:01<00:00,  7.59it/s]
SwarmPy |Final Step| Score : 7658.958320375772: 100%|██████████| 30/30 [00:07<00:00,  4.17it/s]
SwarmPy

In [13]:
px.line(np.std(scores, axis=0))

# plt.plot(scores_w_daemon)
# plt.xlabel('Iterations')
# plt.ylabel('Gap between best solution and optimal solution')
# plt.show()